## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,54.48,98,100,5.19,overcast clouds
1,1,Buala,SB,-8.1450,159.5921,82.04,77,84,2.42,light rain
2,2,Srednekolymsk,RU,67.4500,153.6833,-40.90,100,61,2.64,broken clouds
3,3,Victoria,HK,22.2855,114.1577,65.32,66,98,1.01,overcast clouds
4,4,Fairbanks,US,64.8378,-147.7164,-22.02,73,1,0.00,haze


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Buala,SB,-8.1450,159.5921,82.04,77,84,2.42,light rain
5,5,Vaini,TO,-21.2000,-175.2000,82.56,78,40,10.36,light rain
18,18,Puerto Ayora,EC,-0.7393,-90.3518,75.16,83,71,8.41,broken clouds
20,20,Saint-Philippe,RE,-21.3585,55.7679,78.35,83,26,10.76,scattered clouds
22,22,Rikitea,PF,-23.1203,-134.9692,75.99,72,23,10.16,few clouds
24,24,Hithadhoo,MV,-0.6000,73.0833,81.16,77,79,11.70,broken clouds
26,26,Cabo San Lucas,MX,22.8909,-109.9124,86.50,49,20,3.00,few clouds
29,29,Omboue,GA,-1.5746,9.2618,78.57,89,99,8.55,overcast clouds
31,31,Faanui,PF,-16.4833,-151.7500,81.36,68,17,16.49,few clouds
35,35,San Cristobal,VE,7.7669,-72.2250,78.98,46,64,4.68,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buala,SB,82.04,light rain,-8.1450,159.5921,
5,Vaini,TO,82.56,light rain,-21.2000,-175.2000,
18,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
20,Saint-Philippe,RE,78.35,scattered clouds,-21.3585,55.7679,
22,Rikitea,PF,75.99,few clouds,-23.1203,-134.9692,
24,Hithadhoo,MV,81.16,broken clouds,-0.6000,73.0833,
26,Cabo San Lucas,MX,86.50,few clouds,22.8909,-109.9124,
29,Omboue,GA,78.57,overcast clouds,-1.5746,9.2618,
31,Faanui,PF,81.36,few clouds,-16.4833,-151.7500,
35,San Cristobal,VE,78.98,broken clouds,7.7669,-72.2250,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        continue   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buala,SB,82.04,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
5,Vaini,TO,82.56,light rain,-21.2000,-175.2000,Keleti Beach Resort
18,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Saint-Philippe,RE,78.35,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
22,Rikitea,PF,75.99,few clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
693,Road Town,VG,82.49,few clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
696,Vao,NC,76.32,broken clouds,-22.6667,167.4833,Hotel Kou-Bugny
697,Mandera,KE,84.07,few clouds,3.9366,41.8670,Mwalimu Guest mandera
699,Koupela,BF,81.32,clear sky,12.1794,-0.3517,AUBERGE MAQUIS RESTO CAFE LE ROUTIER PLUS


In [10]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))